In [26]:
# 2025/6/25
# zhangzhong
# https://python.langchain.com/docs/tutorials/agents/
# TODO: 今天把RAG部分给看完就行了
# https://python.langchain.com/docs/tutorials/rag/
# https://python.langchain.com/docs/tutorials/qa_chat_history/
# 把这些给看完就行 大概2个小时

In [27]:
# LangChain supports the creation of agents, or systems that use LLMs as reasoning engines 
# to determine which actions to take and the inputs necessary to perform the action. 
# After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, 
# or whether it is okay to finish. This is often achieved via tool-calling.

In [28]:
# # Import relevant functionality
# from langchain.chat_models import init_chat_model
# # I need to find a better search engine
# # https://python.langchain.com/docs/integrations/tools/#search
# # tavily 有免费额度 还是用吧 毕竟这是一个tutorial
# from langchain_tavily import TavilySearch
# from langgraph.checkpoint.memory import MemorySaver # you chould change to persistance storage, such as redis, sqlite, etc.
# from langgraph.prebuilt import create_react_agent

# # Create the agent
# memory = MemorySaver()
# model = init_chat_model("anthropic:claude-3-5-sonnet-latest")
# search = TavilySearch(max_results=2)
# tools = [search]
# agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [29]:
from dotenv import load_dotenv

load_dotenv()

True

In [30]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in SF', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1750821010, 'localtime': '2025-06-24 20:10'}, 'current': {'last_updated_epoch': 1750820400, 'last_updated': '2025-06-24 20:00', 'temp_c': 13.9, 'temp_f': 57.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 10.7, 'wind_kph': 17.3, 'wind_degree': 251, 'wind_dir': 'WSW', 'pressure_mb': 1017.0, 'pressure_in': 30.03, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 83, 'cloud': 75, 'feelslike_c': 12.5, 'feelslike_f': 54.5, 'windchill_c': 10.6, 'windchill_f': 51.1, 'heatindex_c': 12.0, 'heatindex_f': 53.6, 'dewpoint_c': 11.1, 

In [31]:
# https://python.langchain.com/docs/integrations/chat/deepseek/
from langchain_deepseek import ChatDeepSeek

model = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [32]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hello! 😊 How can I help you today?'

In [33]:
# In order to enable that we use .bind_tools to give the language model knowledge of these tools
model_with_tools = model.bind_tools(tools)

In [34]:
query = "Search for the weather in SF"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

# important!!!
# We can see that there's now no text content, but there is a tool call! It wants us to call the Tavily Search tool.
# This isn't calling that tool yet - it's just telling us to. In order to actually call it, we'll want to create our agent.
# 大模型根据我们的提问，决定要调用search tool
# 然后他通过response告诉了我们这个信息
# 我们拿着大模型的返回值去调用search tool
# 然后把search tool的返回值在返回给大模型
# 然后大模型在整理一份回答给我们
# 这就是一次简单的function calling的过程

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'current weather in San Francisco', 'search_depth': 'basic'}, 'id': 'call_0_4d97f197-c2ed-4e13-8bde-12f8572dbeb8', 'type': 'tool_call'}]


In [35]:
# https://python.langchain.com/docs/tutorials/agents/#create-the-agent
# langgraph is backed by a low-level, highly controllable API in case you want to modify the agent logic.

from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)



In [36]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hello! How can I assist you today? 😊


In [37]:
input_message = {"role": "user", "content": "Search for the weather in SF"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()
# 可以很明显的看到，所有的消息都是一次性输出出来的

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_0_a3d32716-9b3e-449b-a186-7350fde39a65)
 Call ID: call_0_a3d32716-9b3e-449b-a186-7350fde39a65
  Args:
    query: current weather in San Francisco
    search_depth: basic
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather in San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1750824341, 'localtime': '2025-06-24 21:05'}, 'current': {'last_updated_epoch': 1750824000, 'last_updated': '20

In [38]:
# https://python.langchain.com/docs/tutorials/agents/#streaming-messages
# If the agent executes multiple steps, this may take a while. 
# To show intermediate progress, we can stream back messages as they occur.

for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()
    # 看来每个step还是会返回所有的消息，所以需要拿最后一个

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_0_6072d3ee-cda3-46db-9d60-6f22c1282267)
 Call ID: call_0_6072d3ee-cda3-46db-9d60-6f22c1282267
  Args:
    query: current weather in San Francisco
    search_depth: basic
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather in San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1750824341, 'localtime': '2025-06-24 21:05'}, 'current': {'last_updated_epoch': 1750824000, 'last_updated': '20

In [39]:
# https://python.langchain.com/docs/tutorials/agents/#streaming-tokens
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    # TODO: 不知道该怎么把这两种模式结合起来
    # 既展示agent的决策过程，又可以stream token
    if metadata["langgraph_node"] == "agent":
        print(step.text(), end="")
    elif metadata["langgraph_node"] == "tool":
        print(f"\nTool call: {step.tool_call_id}")
        print(f"Tool name: {step.tool_name}")
        print(f"Tool input: {step.tool_input}")
        print(f"Tool output: {step.tool_output}")
        print(f"Tool error: {step.tool_error}")
        print(f"Tool result: {step.tool_result}")
        print(f"Tool result: {step.tool_result}")

The current weather in San Francisco is partly cloudy with a temperature of 57°F (13.9°C). The wind is blowing from the WSW at 9.8 mph (15.8 kph), and the humidity is at 83%. The feels-like temperature is 54.8°F (12.6°C).

For more details, you can check [Weather API](https://www.weatherapi.com/).

In [40]:
# https://python.langchain.com/docs/tutorials/agents/#adding-in-memory
# As mentioned earlier, this agent is stateless
# This means it does not remember previous interactions.
# pass a checkpointer, and the thread id

In [41]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

input_message = {"role": "user", "content": "Hi, I'm Bob!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob!
================================== Ai Message ==================================

Hi, Bob! How can I assist you today?


In [44]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="messages"
):
    print(step)

(AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run--d67a9d19-809d-4971-83cb-fc69cb341f49'), {'thread_id': 'abc123', 'langgraph_step': 10, 'langgraph_node': 'agent', 'langgraph_triggers': ('branch:to:agent',), 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'agent:06c51c7a-15a0-feba-b69b-ff359cdfe82f', 'checkpoint_ns': 'agent:06c51c7a-15a0-feba-b69b-ff359cdfe82f', 'ls_provider': 'openai', 'ls_model_name': 'deepseek-chat', 'ls_model_type': 'chat', 'ls_temperature': 0.0})
(AIMessageChunk(content='Your', additional_kwargs={}, response_metadata={}, id='run--d67a9d19-809d-4971-83cb-fc69cb341f49'), {'thread_id': 'abc123', 'langgraph_step': 10, 'langgraph_node': 'agent', 'langgraph_triggers': ('branch:to:agent',), 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'agent:06c51c7a-15a0-feba-b69b-ff359cdfe82f', 'checkpoint_ns': 'agent:06c51c7a-15a0-feba-b69b-ff359cdfe82f', 'ls_provider': 'openai', 'ls_model_name': 'deeps